# graph

This builds a Mermaid graph of the tasks.

In [ ]:
import json
import sys
from collections import defaultdict
from importlib.util import module_from_spec, spec_from_file_location
from pathlib import Path
from uuid import uuid4

import importnb
from doit.cmd_base import ModuleTaskLoader, get_loader
from doit.cmd_list import List as ListCmd
from doit.dependency import Dependency, SqliteDB

In [ ]:
NODE_STYLES = {
    "py": "fill:#4b8bbe,color:#ffe873;",
    "jupyter": "fill:#f37626,color:#ffe873;",
    "js": "fill:#f0db4f,color:#000;",
    "git": "fill:#f1502f,color:#fff;",
    "hack": "fill:#000,color:#fff;",
}
NODE_SHAPES = {
    "py": ("[", "]"),
    "jupyter": ("([", "])"),
    "js": ("{{", "}}"),
    "git": ("[(", ")]"),
    "hack": ("((", "))"),
}
STATUS = {
    "up-to-date": "✅",
    "run": "💭",
    "error": "❌",
}
GRAPH_OPTIONS = {"flowchart": {"defaultRenderer": "elk"}}
BLURB = """
The graph below shows the current state of the executed graph of tasks to go from
<code>git clone</code> to <code>jupyter lite build</code>.

<blockquote>
Some tasks, such as extracting <code>conda</code> dependencies from <code>pyproject.toml</code>
files, have been elided to highlight the relationships between repositories.
</blockquote>
"""

In [ ]:
def all_tasks():
    with importnb.Notebook():
        from jupyak.tasks._self import ROOT
    old_sys_path = [*sys.path]
    mod_name = f"""__dodo__{str(uuid4()).replace("-", "_")}"""
    dodo_module = None
    try:
        sys.path += [str(ROOT)]
        spec = spec_from_file_location(mod_name, ROOT / "dodo.py")
        if spec:
            dodo_module = module_from_spec(spec)

            if dodo_module is None or spec.loader is None:  # pragma: no cover
                return []
            sys.modules[mod_name] = dodo_module
            spec.loader.exec_module(dodo_module)
    finally:
        sys.path = old_sys_path

    loader = get_loader({}, task_loader=ModuleTaskLoader(dodo_module.__dict__))
    cmd = ListCmd(loader)
    tasks: Tasks = loader.load_tasks(cmd, [])
    tracker = Dependency(SqliteDB, str(ROOT / ".doit.db"))
    return tasks, tracker

In [ ]:
def filtered_tasks(
    ignore_prefixes=["bootstrap", "self", "shave:env", "shave:js:not-a-package"],
    ignore_substrings=[":deps:"],
):
    tasks, tracker = all_tasks()
    is_ignored = lambda t: (
        any(t.name.startswith(i) for i in ignore_prefixes)
        or any(substr in t.name for substr in ignore_substrings)
    )
    return [t for t in tasks if not is_ignored(t)], tracker

In [ ]:
def mermaid_preamble(direction="BT", title=None, description=None):
    mmd = [
        f"""graph {direction}""",
        f"""%%{{init: {json.dumps(GRAPH_OPTIONS)} }}%%""",
    ]
    # doesn't work with elk :(
    # if title:
    #     mmd += [f"accTitle: {title}"]
    # if description:
    #     mmd += [f"accDescr: {description}"]
    mmd += [
        *[f"classDef {cls} {style}" for cls, style in NODE_STYLES.items()],
    ]
    return mmd

In [ ]:
def build_legend() -> list[str]:
    return [
        *mermaid_preamble(
            direction="TB",
            title="Legend",
            description="Examples of graph node styles used in the full task diagram",
        ),
        "subgraph task types",
        *[
            f"""{cls}{NODE_SHAPES[cls][0]}{cls}{NODE_SHAPES[cls][1]}:::{cls}"""
            for cls in NODE_STYLES
        ],
        "end",
        "subgraph task status",
        *[f"""{status}["{emoji} {status}"]""" for status, emoji in STATUS.items()],
        "end",
    ]

In [ ]:
def build_graph(direction="LR", title=None, description=None) -> list[str]:
    title = title or "Task Graph"
    description = "A flow chart of the jupyak task graph"
    tasks, tracker = filtered_tasks()
    by_name = {t.name: t for t in tasks}
    file_dep = {t.name: set(t.file_dep) for t in tasks}
    targets = {t.name: set(t.targets) for t in tasks}

    nl = "\n"
    mmd = [*mermaid_preamble(direction=direction, title=title, description=description)]

    groups = defaultdict(list)

    for task in tasks:
        name = task.name
        bits = name.split(":")
        if bits[0] == "shave" and len(bits) >= 3:
            group = bits[1] if bits[1] == "lite" else bits[2]
            groups[group] += [task]
        elif bits[0] == "jupyterlab" and len(bits) > 1:
            groups["jupyterlab"] += [task]
        mmd += [
            *[
                f"{ot} --> {name}"
                for ot, tgts in targets.items()
                if tgts & file_dep[task.name]
            ],
        ]
    for group, grouped in groups.items():
        mmd += [f"""subgraph {group}"""]
        for task in grouped:
            bits = task.name.split(":")
            label = " ".join(bits[3:] if bits[3:] else bits[2:])
            status = tracker.get_status(task, tasks).status
            status = STATUS.get(status, f"`{status}`")
            cls = bits[1]
            if "labext" in bits or ("lite" in bits and "git" not in bits):
                cls = "jupyter"
            if "sweep" in bits:
                cls = "hack"
            shape = f"""{NODE_SHAPES[cls][0]}"`{status} **{label}**`"{NODE_SHAPES[cls][1]}"""
            mmd += [f"""  {task.name}{shape}:::{cls}"""]
        mmd += ["end"]

    return mmd

In [ ]:
def div_wrappers(chunks: list[str], dom_id=None, show_zoom=False, legend=False) -> str:
    dom_id = dom_id or f"id-{uuid4()}"
    chunks = [
        f"""<div class="jp-Mermaid" id="{dom_id}">""",
        zoom(f"{dom_id}-") if show_zoom else "",
        """<div class="mermaid">""",
        *chunks,
        "</div>",
        "</div>",
    ]
    return "\n".join(chunks)

In [ ]:
def zoom(dom_id=""):
    html = ["<label>Zoom: </label>"]
    html += [
        f"""<input type="radio" id="{dom_id}svg-zoom-{i}" name="{dom_id}svg-zoom" {"checked" if i == 1 else "" }>"""
        f"""<label for="svg-zoom-{i}"> {i}x</label>"""
        for i in [1, 2, 4, 8]
    ]
    return "\n".join(html)

In [ ]:
def write_graph(dest: Path, **options):
    dest.parent.mkdir(parents=True, exist_ok=True)
    chunks = [
        "---",
        "html_theme.sidebar_secondary.remove: true",
        "---",
        "",
        "# task graph",
        "",
        BLURB,
        "",
        div_wrappers(build_legend(), dom_id="task-graph-legend"),
        "",
        div_wrappers(build_graph(**options), dom_id="task-graph", show_zoom=True),
    ]
    dest.write_text("\n".join(chunks), encoding="utf-8")